In [1]:
from numpy.random import seed
seed(1)

In [2]:
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Flatten, CuDNNLSTM

Using TensorFlow backend.


In [3]:
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Flatten, CuDNNLSTM
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping

import numpy as np

from numpy import argmax
import pandas as pd
df = pd.read_excel('80.xlsx',  header=None)

#df = df.sort_values(by = 2, ascending=False)

df.dropna(inplace=True)

#df.loc[:,0] = first_col(df.loc[:,0])

df.drop(labels=[119974], inplace=True)

df[0]= df[0].apply(lambda x: '_'.join([i for i in x]))

df[1] = df[1].apply(lambda x: x.replace(" ","_"))

In [4]:
df['len0'] = [len(x) for x in df[0]]
df['len1'] = [len(x) for x in df[1]]

In [5]:
#df = df[df[2] > 3]

In [6]:
df2 = df.copy()

In [7]:
df2 = df2[df2['len0'] == df2['len1']]

In [8]:
max(df2['len1'])

39

In [9]:
eng_alph = '_abcdefghijklmnopqrstuvwxyz'
rus_alph = '_абвгдеёжзийклмнопрстуфхцчшщыьъэюя'

eng_c2i = dict((c, i) for i, c in enumerate(eng_alph))
eng_i2c = dict((i, c) for i, c in enumerate(eng_alph))

rus_c2i = dict((c, i) for i, c in enumerate(rus_alph))
rus_i2c = dict((i, c) for i, c in enumerate(rus_alph))

In [10]:
df = df.reset_index(drop = True)

In [11]:
max_char_rus = max([len(i) for i in df2[0]])

max_char_eng = max([len(i) for i in df2[1]])

In [12]:
max_len_alph = max(len(eng_alph), len(rus_alph))

In [13]:
df2.drop(df2[df2.loc[:,1].str.contains('\\?')].index, inplace=True)

df2.drop(df2[df2.loc[:,1].str.contains('ў')].index, inplace=True)

df2.drop(df2[df2.loc[:,1].str.contains('’')].index, inplace=True)
df2.drop(df2[df2.loc[:,0].str.contains('’')].index, inplace=True)

df2.drop(df2[df2.loc[:,1].str.contains('і')].index, inplace=True)

df2.drop(df2[df2.loc[:,1].str.contains('ј')].index, inplace=True)

df2.drop(df2[df2.loc[:,1].str.contains('ќ')].index, inplace=True)

df2.drop(df2[df2.loc[:,1].str.contains('ћ')].index, inplace=True)

df2.drop(df2[df2.loc[:,0].str.contains('`')].index, inplace=True)

df2[df2.loc[:,0].str.contains('`')]

In [22]:
df2.reset_index(drop=True, inplace=True)

In [23]:
rus_enc = np.zeros((len(df2),max_char_rus,max_len_alph),dtype = 'uint8')
#i - row
#j - symbol position
#k - character encoding

for i in range(len(df2)):
    for j in range(max_char_rus):
        for k in range(max_len_alph):
            if len(df2.iloc[i,1])>j and k==rus_c2i[df2.iloc[i,1][j]]:
                rus_enc[i,j,k] = 1
            else:
                rus_enc[i,j,k] = 0

eng_enc = np.zeros((len(df2),max_char_eng,max_len_alph),dtype = 'uint8')

for i in range(len(df2)):
    for j in range(max_char_eng):
        for k in range(max_len_alph):
            if len(df2.iloc[i,0])>j and k==eng_c2i[df2.iloc[i,0][j]]:
                eng_enc[i,j,k] = 1
            else:
                eng_enc[i,j,k] = 0

In [29]:
np.save('rus_enc.npy', rus_enc)

In [30]:
np.save('eng_enc.npy', eng_enc)

In [25]:
print(eng_enc.shape)
print(rus_enc.shape)

(70211, 39, 34)
(70211, 39, 34)


In [52]:
model = Sequential()

model.add(CuDNNLSTM(34, input_shape = rus_enc.shape[1:],  return_sequences = True))
#model.add(Dropout(0.1))
#model.add(CuDNNLSTM(34,  return_sequences = True))


#model.add(Dropout(0.1))
#model.add(Dense(34, activation = 'sigmoid'))
#model.add(CuDNNLSTM(34,  return_sequences = True))
model.add(Dense(34, activation = 'softmax'))
#model.add(LSTM(128))

#model.add(Dropout(0.2))


model.compile(loss = 'categorical_crossentropy',optimizer= 'Adam', metrics = ['accuracy'])

model.fit(rus_enc, eng_enc, epochs=50, batch_size =1152,  validation_split = 0.3)

Train on 49147 samples, validate on 21064 samples
Epoch 1/50
49147/49147 [==============================] - 4s 74us/step - loss: 1.0202 - acc: 0.7546 - val_loss: 0.8399 - val_acc: 0.8297
Epoch 2/50
49147/49147 [==============================] - 2s 31us/step - loss: 0.7743 - acc: 0.8296 - val_loss: 0.7318 - val_acc: 0.8297
Epoch 3/50
49147/49147 [==============================] - 2s 32us/step - loss: 0.7063 - acc: 0.8296 - val_loss: 0.6792 - val_acc: 0.8297
Epoch 4/50
49147/49147 [==============================] - 2s 31us/step - loss: 0.6510 - acc: 0.8296 - val_loss: 0.6158 - val_acc: 0.8297
Epoch 5/50
49147/49147 [==============================] - 2s 31us/step - loss: 0.5675 - acc: 0.8301 - val_loss: 0.5137 - val_acc: 0.8436
Epoch 6/50
49147/49147 [==============================] - 2s 32us/step - loss: 0.4803 - acc: 0.8688 - val_loss: 0.4503 - val_acc: 0.8746
Epoch 7/50
49147/49147 [==============================] - 2s 32us/step - loss: 0.4252 - acc: 0.8813 - val_loss: 0.3994 - val_acc

In [46]:
#model.save('98acc30step')

In [297]:
bdfr_cols = [x for x in eng_alph]
[bdfr_cols.append("?") for i in range(7)]

def mytranslate(word):
    b = np.zeros((39,34))
    for ix, char in enumerate(word):
        b[ix, rus_c2i[char]] = 1
    b = b.reshape((1, 39,34))
    predicted = model.predict(b)
    predicted = predicted.reshape(39,34)
    dfr = pd.DataFrame(predicted, columns=bdfr_cols)
    dfr.to_excel(word +'.xlsx')
    dfr = round(dfr)
    answer = []
    for i, _ in dfr.iterrows():
        try:
            answer.append(dfr.loc[i,:][dfr.loc[i,:]==1].index[0])
        except:
            pass
    return ''.join(answer)

In [ ]:
#############################################################################
#DO NOT USE
#############################################################################